In [2]:
import pandas as pd

In [3]:
YEAR = 2023
TARGET = 'customer'

In [4]:
PATH = f'../dataset/imbank_{TARGET.lower()}_data/iMBANK_{TARGET.upper()}_DATA_{YEAR}(K-DigitalTraining).csv'
df = pd.read_csv(PATH, encoding='euc-kr')

In [5]:
customer = df.groupby('고객ID')['기준년월'].nunique()

# 12개 이상의 기준년월을 가진 고객ID 필터링
customer = customer[customer == 12].index

# 필터링된 고객ID에 해당하는 데이터 추출
df_12 = df[df['고객ID'].isin(customer)]

del df

In [6]:
df_12['총자산'] = df_12[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액']].sum(axis=1)

In [7]:
# 분기 계산 (연도와 분기 번호 생성)
df_12['분기'] = ((df_12['기준년월'] - YEAR * 100) // 4) + 1

In [8]:
df_age_q = df_12.groupby(['연령대', '분기'])['총자산'].mean().reset_index()

In [9]:
df_12= pd.merge(df_12, df_age_q, on=['연령대', '분기'], how='left')

In [10]:
df_12.columns = ['기준년월', '고객ID', '연령대', '성별', '고객등급', '자택_시도', '자택_시군구', '수신_요구불예금',
       '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액', '총자산', '분기',
       '연령_분기별_자산평균']

# 분기 초/말의 총자산을 계산
df_12['분기_말_총자산'] = df_12.groupby(['고객ID', '분기'])['총자산'].transform('first')
df_12['분기_초_총자산'] = df_12.groupby(['고객ID', '분기'])['총자산'].transform('last')

# 분기 내 자산 변화량 계산
df_12['자산변화량'] = (df_12['분기_말_총자산'] - df_12['분기_초_총자산'])
df_12['자산변화율'] =  df_12['자산변화량'] / df_12['연령_분기별_자산평균']

In [11]:
df_12

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액,총자산,분기,연령_분기별_자산평균,분기_말_총자산,분기_초_총자산,자산변화량,자산변화율
0,202312,9999998161,60대이상,여,일반등급,대구광역시,달서구,300000,0,0,0,0,0,300000,4,1.023246e+07,300000,300000,0,0.000000
1,202312,9999997589,60대이상,남,일반등급,경상북도,경주시,0,0,0,0,0,0,0,4,1.023246e+07,0,0,0,0.000000
2,202312,9999991579,60대이상,여,일반등급,경상북도,성주군,0,33000000,0,0,0,0,33000000,4,1.023246e+07,33000000,33000000,0,0.000000
3,202312,9999989448,60대이상,남,일반등급,대구광역시,달서구,400000,0,0,0,0,0,400000,4,1.023246e+07,400000,400000,0,0.000000
4,202312,9999988325,60대이상,남,일반등급,대구광역시,달서구,2800000,0,0,0,0,0,2800000,4,1.023246e+07,2800000,2800000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21815827,202301,2455102445,30대,여,일반등급,서울특별시,마포구,0,0,0,0,0,0,0,1,5.688877e+06,10000000,0,10000000,1.757816
21815828,202301,2455101298,30대,여,일반등급,서울특별시,성북구,0,0,0,0,0,0,0,1,5.688877e+06,2500000,0,2500000,0.439454
21815829,202301,2455099113,20대,여,일반등급,서울특별시,강동구,0,0,0,0,0,0,0,1,3.137101e+06,1300000,0,1300000,0.414395
21815830,202301,2455093090,20대,여,일반등급,서울특별시,광진구,0,0,0,0,0,0,0,1,3.137101e+06,1500000,0,1500000,0.478148


In [ ]:
# 변화율에 따라 고객 분류
def classify_change_rate(change_rate):
    if change_rate > 10:
        return '초안전'
    elif change_rate > 0:
        return '안전'
    elif change_rate == 0:
        return '무관심'
    elif change_rate >= -10:
        return '위험'
    else:
        return '고위험'

In [15]:
df_12

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액,총자산,분기,연령_분기별_자산평균,분기_말_총자산,분기_초_총자산,자산변화량,자산변화율
0,202312,9999998161,60대이상,여,일반등급,대구광역시,달서구,300000,0,0,0,0,0,300000,4,1.023246e+07,300000,300000,0,0.000000
1,202312,9999997589,60대이상,남,일반등급,경상북도,경주시,0,0,0,0,0,0,0,4,1.023246e+07,0,0,0,0.000000
2,202312,9999991579,60대이상,여,일반등급,경상북도,성주군,0,33000000,0,0,0,0,33000000,4,1.023246e+07,33000000,33000000,0,0.000000
3,202312,9999989448,60대이상,남,일반등급,대구광역시,달서구,400000,0,0,0,0,0,400000,4,1.023246e+07,400000,400000,0,0.000000
4,202312,9999988325,60대이상,남,일반등급,대구광역시,달서구,2800000,0,0,0,0,0,2800000,4,1.023246e+07,2800000,2800000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21815827,202301,2455102445,30대,여,일반등급,서울특별시,마포구,0,0,0,0,0,0,0,1,5.688877e+06,10000000,0,10000000,1.757816
21815828,202301,2455101298,30대,여,일반등급,서울특별시,성북구,0,0,0,0,0,0,0,1,5.688877e+06,2500000,0,2500000,0.439454
21815829,202301,2455099113,20대,여,일반등급,서울특별시,강동구,0,0,0,0,0,0,0,1,3.137101e+06,1300000,0,1300000,0.414395
21815830,202301,2455093090,20대,여,일반등급,서울특별시,광진구,0,0,0,0,0,0,0,1,3.137101e+06,1500000,0,1500000,0.478148


In [ ]:
df_12.groupby(['연령대', '분기'])['자산변화량'].transform('mean')